**Śledzenie gry w bilard**

Autorzy: Joanna Cicha, Przemysław Łabuń, Maciej Mak

1. Przygotowanie zbioru danych

In [127]:
import cv2
import numpy as np
import os
import glob

from loky import get_reusable_executor

In [128]:
# Reads frames from the video, which will be used as the dataset for tracking and analysis.
def read_video(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    i = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)
        i += 1
    cap.release()
    return frames

# Loads templates that are necessary for object detection in video frames.
def load_templates(template_folder, suffix):
    templates = []
    for filename in os.listdir(template_folder):
        if filename.endswith(suffix):
            path = os.path.join(template_folder, filename)
            template = cv2.imread(path, 0)
            templates.append(template)
    return templates

**2. Wykorzystane techniki**

**2.1. Template Matching** <br>
<br>
Technika do wykrywania określonych obiektów (takich jak kule i łuzy) w ramkach wideo. Polega na użyciu predefiniowanych szablonów – małych obrazów obiektów zainteresowania – i przesuwaniu tych szablonów po docelowej ramce, aby znaleźć dopasowania na podstawie metryki podobieństwa. <br><br>

**2.2. Transformacje Geometryczne i Rysowanie** <br><br>
Do wizualnego podkreślenia wykrytych obiektów i zdarzeń w wideo używane są funkcje transformacji geometrycznych i rysowania. <br><br>

**2.3. Filtrowanie Bliskich Punktów** <br><br>
Aby unikać wielokrotnych wykryć tego samego obiektu, punkty znajdujące się zbyt blisko siebie są filtrowane.<br><br>

**2.4. Przetwarzanie Równoległe** <br><br>
Aby zwiększyć wydajność przetwarzania wielu klatek wideo, szczególnie dla filmów o wysokiej rozdzielczości, używane jest przetwarzanie równoległe.<br><br>

**2.5. Odczytywanie i Zapisywanie Wideo** <br><br>
Polega na ekstrakcji klatek z pliku wideo, a następnie kompilacji przetworzonych klatek z powrotem na wideo. <br><br>


In [129]:
# Filters out points that are too close to each other based on a given threshold.
def filter_close_points(points, threshold):
    keep_points = []
    for point in points:
        if keep_points:
            distances = np.linalg.norm(np.array(keep_points)[:, :2] - point[:2], axis=1)
            if np.all(distances > threshold):
                keep_points.append(point)
        else:
            keep_points.append(point)
    return np.array(keep_points)

In [130]:
def draw_rectangles(frame, matched_points):
    for (x, y, w, h) in matched_points:
        cv2.rectangle(frame, (x - w // 2, y - h // 2), (x + w // 2, y + h // 2), (255, 0, 0), 2)
    return frame

In [131]:
# Matches given templates with the frame content above a certain threshold and returns the center points of matched areas.
def match_templates(frame, templates, threshold=0.8, n=1):
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    matched_points = []
    for template in templates:
        w, h = template.shape[::-1]
        res = cv2.matchTemplate(gray_frame, template, cv2.TM_CCOEFF_NORMED)
        loc = np.where(res >= threshold)
        for pt in zip(*loc[::-1]):  # Reverse tuple to get (x, y) coordinates
            center_point = [pt[0] + w // 2, pt[1] + h // 2, w, h]
            matched_points.append(center_point)
    matched_points = np.array(matched_points)
    matched_points = filter_close_points(matched_points, 50).tolist()
    return matched_points


In [132]:
# Identifies initial positions of pool sockets by matching templates from the first frame.
def get_initial_socket_positions(frame, pocket_template_folder, suffix='.png'):
    pocket_templates = load_templates(pocket_template_folder, suffix)
    pocket_positions = match_templates(frame, pocket_templates, 0.85)
    return pocket_positions


In [133]:
# Detects collisions between the white ball and other balls.
def detect_collision(white_ball_position, other_balls_positions, ball_radius):
    collisions = []
    for index, pos in enumerate(other_balls_positions):
        distance = np.linalg.norm(np.array(white_ball_position[:2]) - np.array(pos[:2]))
        print(f"Distance between balls {index} and white ball: {distance}")
        total_radius = 2 * ball_radius 
        if distance <= total_radius:
            collisions.append(index)
    return collisions

In [134]:
# Extracts socket positions from the first frame of a video sequence.
def get_socket_positions_from_first_frame(frames, pocket_template_folder):
    if not frames:
        return None, [] 
    first_frame = frames[0]
    pocket_templates = load_templates(pocket_template_folder, '.png')
    pocket_positions = match_templates(first_frame, pocket_templates, 0.75)
    return first_frame, pocket_positions

In [135]:
# Calculates positions for the middle top and bottom sockets based on the leftmost and rightmost detected positions.
def calculate_middle_sockets(pocket_positions):
    leftmost, rightmost = pocket_positions[0], pocket_positions[-1]
    middle_top = ((leftmost[0] + rightmost[0]) // 2, min(leftmost[1], rightmost[1]))
    middle_bottom = ((leftmost[0] + rightmost[0]) // 2, max(leftmost[1], rightmost[1]))
    return middle_top, middle_bottom

In [136]:
# Main function to process each frame, applying template matching, drawing results, and detecting objects' interactions.
def process_frame(frame_args):
    frame, i, ball_template_folder, socket_positions = frame_args
    
    ball_templates = load_templates(ball_template_folder, '.png')
    
    ball_positions = match_templates(frame, ball_templates, 0.9)

    frame_with_balls = draw_rectangles(frame, ball_positions)
    frame_with_pockets = draw_rectangles(frame_with_balls, socket_positions)

    if len(socket_positions) >= 2:
        middle_top, middle_bottom = calculate_middle_sockets(socket_positions)
        cv2.circle(frame_with_pockets, middle_top, 10, (0, 255, 255), -1)
        cv2.circle(frame_with_pockets, middle_bottom, 10, (0, 255, 255), -1)

    message = None
    for ball_pos in ball_positions:
        for pocket_pos in socket_positions:
            x, y, w, h = pocket_pos
            pocket_center = (x, y)
            distance = np.linalg.norm(np.array(ball_pos[:2]) - np.array(pocket_center))
            if distance < 100:
                message = f"Ball in the socket at position: {ball_pos[:2]} in frame {i}"
                cv2.putText(frame_with_pockets, message, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                break 
        if message:
            break 
    img = frame_with_balls
    cv2.imwrite(f"results/frame_{i}.jpg", img)

    return ball_positions

ball_radius = 11
ball_template_folder = 'template'
pocket_template_folder = 'sockets'
video_path = 'bilard.mp4'
frames = read_video(video_path)
initial_frame, initial_socket_positions = get_socket_positions_from_first_frame(frames, pocket_template_folder)

if initial_frame is not None:
    executor = get_reusable_executor(max_workers=16, timeout=2)
    frame_args = [(frame, i, ball_template_folder, initial_socket_positions) for i, frame in enumerate(frames)]
    results = executor.map(process_frame, frame_args)
    frames_ball_positions = list(results)
else:
    print("No frames available to process.")

In [137]:
# Combines processed frames into a video file.
def create_video_from_images(image_folder, output_video, frame_rate=30):
    image_files = glob.glob(os.path.join(image_folder, '*.jpg'))
    image_files = sorted(image_files, key=lambda x: int(x.split("_")[1].split(".")[0]))

    first_image = cv2.imread(image_files[0])
    height, width, layers = first_image.shape

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video, fourcc, frame_rate, (width, height))

    for filename in image_files:
        img = cv2.imread(filename)
        out.write(img)

    out.release()
    cv2.destroyAllWindows()

create_video_from_images('results', 'movie.mp4')